In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install fasttext

     |████████████████████████████████| 71kB 5.6MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3095092 sha256=0006c789d703062cb58942bd2e96d942c92d5281453c8c49ca67ec653fcd4558
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
pip install neattext

     |████████████████████████████████| 112kB 9.4MB/s 


In [ ]:
import pandas as pd
import string #Opérations courantes sur les chaînes
import re   # pour utiliser  des expressions régulières ex:$
import nltk #boites outils ntk
from sklearn.model_selection import train_test_split
import neattext as nt
import neattext.functions as nfx
from neattext import TextCleaner as tc

In [ ]:
#Cleaning & Preprocessing
def process(text):
# la normalisation
    regex = re.compile(r'[إأٱآا]') #sauvegarder l'expression réguliere (r=expression reguliere)
    text = re.sub(regex, 'ا', str(text)) #remblacer des expressions par d'autres dans varia texte
    regex = re.compile(r'[ى]')
    text = re.sub(regex, 'ي', str(text))
    regex = re.compile(r'[ؤئ]')
    text = re.sub(regex, 'ء', str(text))
    text = re.sub('\n', ' ', str(text))
    text = re.sub(' و ', ' ', str(text))
    text = re.sub(r'\s*[A-Za-z]+\b', '' , text)
    text = text.rstrip()
#supprimer les diacritiques
    arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)   #verbose: séparer visuellement les expre et ajouter des commentaires
    text = re.sub(arabic_diacritics, '', text)
    

#supprimer les ponctuations
    #liste des ponctuations arabe et anglais qu'on veut supprimer
    arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''    # utiliser ''' :multiples caractére de chaine
    english_punctuations = string.punctuation              #(!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~)
    punctuations_list = arabic_punctuations + english_punctuations
    #table de mappage
    translator = str.maketrans('','', punctuations_list) # le 1er elem a remplacer/le 2eme le remplacement,3eme=supprimer
    text= text.translate(translator) #chaine ou les caractere sont remplacé par d'autres dns table de mappage

#supprimer les chaines répétés
    text=re.sub(r'(.)\1+', r'\1', str(text)) 

#supprimer les mots vides
    stopwords=nltk.corpus.stopwords.words('arabic') #les mots vides en arabe
    txt=[word for word in text if word not in stopwords]

#supprimer les émojis
    myre = re.compile(u'['
    u'\U0001F300-\U0001F64F'
    u'\U0001F680-\U0001F6FF'     
    u'\u2600-\u26FF\u2700-\u27BF]+', re.UNICODE)
    text=myre.sub('', text)
    
    
    return text

In [ ]:
filename='new_db.xlsx'

In [ ]:
#Importer le fichier excel
data=pd.read_excel(filename) #filename:le chemin d'acces au fichier excel

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Cleaning DB
data['TWEET'] = data['TWEET'].apply(process)
data['TWEET'] = data['TWEET'].apply(nfx.remove_hashtags)
data['TWEET'] = data['TWEET'].apply(nfx.remove_userhandles)

In [ ]:
index=[]
for i in range(len(data)):
  if data.iloc[i].TWEET=='':
    index.append(i)

for i in index:
  data=data.drop(i)
data = data.reset_index().drop('index',axis=1)

In [ ]:
#Affichage des premiers elts
data.head()

,TWEET,LABEL
0,الاوليمبياد الجايه هكون لسه ف الكليه,0
1,عجز الموازنه وصل ل937 من الناتج المحلي يعني لس...,1
2,كتنا نيله ف حظنا الهباب,2
3,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,3
4,الاوليمبياد نظامها مختلف ومواعيد المونديال مكا...,0


In [ ]:
#Affichage des derniers elts
data.tail()

,TWEET,LABEL
29947,اتمني تعملو بطاقات خاصة او ذهبية لعملاء الداءم...,3
29948,المشويات عندهم اطيب مشويات اكلتها بحياتي والاس...,3
29949,المطعم متاز لغايه من ناحيه تجهيزات المطعم وايض...,3
29950,جينا عندهم الساعة 10 باليل مافي احد اتبرع يشوف...,1
29951,اذا كان الي بالشارع التجاري اوه ماشاء اله واله...,3


In [ ]:
#Remplacer chaque label par un entier
encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
        encode_dict[x]=len(encode_dict)
    return encode_dict[x]

data['LABEL'] = data['LABEL'].apply(lambda x: encode_cat(x))

In [ ]:
#Split data into random train(70%) and test(30%) subsets
text_train, text_test, label_train, label_test = train_test_split(data.TWEET,data.LABEL,test_size=0.20, random_state=42)
print(text_train.shape,text_test.shape,label_train.shape,label_test.shape)  

(23961,) (5991,) (23961,) (5991,)


# FASTTEXT

# Preparation du model

In [ ]:
import fasttext

In [ ]:
# load AraVec Spacy model
%cd /content/drive/MyDrive/fasttext
ft = fasttext.load_model('cc.ar.300.bin')
%cd ..
%cd ..
%cd ..

/content/drive/MyDrive/fasttext
/content/drive/My Drive
/content/drive
/content


# Machine learning

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import numpy as np

In [ ]:
def get_vect(word, model):
    try:
        return model[word]
    except KeyError:
        return np.zeros((model.get_dimension(),))

def sum_vectors(phrase, model):
    sen=phrase.split()
    return sum([get_vect(w, model) for w in sen])

def fasttext_features_pred(phrase, model):
    sen=phrase.split()
    somme_vec=sum([get_vect(w, model) for w in sen])
    if isinstance(somme_vec,int):
        return np.zeros((model.get_dimension(),))
    else:
        return somme_vec

def fasttext_features(X, model):
    liste=[]
    for p in X:
        somme_vec=sum_vectors(p, model)
        if isinstance(somme_vec,int):
            liste.append(np.zeros((model.get_dimension(),)))
        else:
            liste.append(somme_vec)
    return np.vstack(liste)

In [ ]:
# Get the sentence embeddings for the train and test dataset
ft_train_feat = fasttext_features(text_train, ft)
ft_test_feat = fasttext_features(text_test, ft)

In [ ]:
# create the model, train it, print scores
lrft = LogisticRegression(solver='liblinear',max_iter=3000)
lrft.fit(ft_train_feat, label_train)
tag_pred = lrft.predict(ft_test_feat)
print(classification_report(label_test, tag_pred))

              precision    recall  f1-score   support

           0       0.59      0.65      0.62       314
           1       0.67      0.57      0.62      1184
           2       0.54      0.38      0.45       701
           3       0.69      0.90      0.78      2434
           4       0.45      0.21      0.29       470
           5       0.65      0.50      0.56       381
           6       0.73      0.66      0.69       231
           7       0.84      0.80      0.82       276

    accuracy                           0.67      5991
   macro avg       0.64      0.58      0.60      5991
weighted avg       0.65      0.67      0.65      5991



In [ ]:
# create the model, train it, print scores
SVM=svm.SVC()
SVM.fit(ft_train_feat,label_train)
tag_pred = SVM.predict(ft_test_feat)
print(classification_report(label_test, tag_pred))

              precision    recall  f1-score   support

           0       0.55      0.66      0.60       314
           1       0.67      0.67      0.67      1184
           2       0.55      0.37      0.44       701
           3       0.69      0.91      0.78      2434
           4       0.58      0.16      0.25       470
           5       0.76      0.51      0.61       381
           6       0.78      0.61      0.68       231
           7       0.93      0.67      0.78       276

    accuracy                           0.68      5991
   macro avg       0.69      0.57      0.60      5991
weighted avg       0.67      0.68      0.65      5991



In [ ]:
# create the model, train it, print scores
rf=RandomForestClassifier()
rf.fit(ft_train_feat,label_train)
tag_pred = rf.predict(ft_test_feat)
print(classification_report(label_test, tag_pred))

              precision    recall  f1-score   support

           0       0.54      0.57      0.55       314
           1       0.74      0.78      0.76      1184
           2       0.76      0.54      0.63       701
           3       0.77      0.93      0.84      2434
           4       0.77      0.51      0.61       470
           5       0.73      0.59      0.65       381
           6       0.77      0.54      0.63       231
           7       0.88      0.55      0.67       276

    accuracy                           0.75      5991
   macro avg       0.74      0.63      0.67      5991
weighted avg       0.75      0.75      0.74      5991



In [ ]:
# create the model, train it, print scores
nb=GaussianNB()
nb.fit(ft_train_feat,label_train)
tag_pred = nb.predict(ft_test_feat)
print(classification_report(label_test, tag_pred))

              precision    recall  f1-score   support

           0       0.10      0.88      0.18       314
           1       0.47      0.29      0.36      1184
           2       0.18      0.05      0.07       701
           3       0.60      0.30      0.40      2434
           4       0.16      0.02      0.03       470
           5       0.28      0.07      0.11       381
           6       0.25      0.23      0.24       231
           7       0.12      0.34      0.18       276

    accuracy                           0.26      5991
   macro avg       0.27      0.27      0.20      5991
weighted avg       0.41      0.26      0.28      5991



# Deep Learning

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import *
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
import numpy as np
from sklearn.metrics import classification_report

In [ ]:
#create a word-to-index dictionary
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_test = tokenizer.texts_to_sequences(text_test)

In [ ]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

#Set the maximum size of each list to 82
maxlen = 82

#Add 0 at the end of the list until it reaches the max length
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
#Create an embedding matrix where each row number will correspond to the index of the word in the corpus
embedding_matrix = np.zeros((vocab_size, 300))
for word, index in tokenizer.word_index.items():
    embedding_vector = ft[word]
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
#--------------------------------------------------------------------------------------------------------------

In [ ]:
model1 = Sequential()
embedding_layer = Embedding(vocab_size,300 , weights=[embedding_matrix], input_length=maxlen , trainable=False)
model1.add(embedding_layer)

# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
model1.add(GRU(256, activation='sigmoid',recurrent_activation='sigmoid' , return_sequences=True))

# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
model1.add(SimpleRNN(128))


model1.add(Dense(8, activation='sigmoid'))
model1.compile(optimizer='Adamax', loss='sparse_categorical_crossentropy', metrics=['acc'])
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 82, 300)           19754700  
_________________________________________________________________
gru (GRU)                    (None, 82, 256)           428544    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               49280     
_________________________________________________________________
dense (Dense)                (None, 8)                 1032      
Total params: 20,233,556
Trainable params: 478,856
Non-trainable params: 19,754,700
_________________________________________________________________


In [ ]:
history = model1.fit(X_train, label_train, batch_size=128, epochs=30, verbose=1, validation_split=0.2)

score = model1.evaluate(X_test, label_test, verbose=1)

Epoch 1/30
150/150 [==============================] - 60s 246ms/step - loss: 1.7910 - acc: 0.3832 - val_loss: 1.7262 - val_acc: 0.4129
Epoch 2/30
150/150 [==============================] - 36s 243ms/step - loss: 1.7261 - acc: 0.4172 - val_loss: 1.6913 - val_acc: 0.4356
Epoch 3/30
150/150 [==============================] - 34s 230ms/step - loss: 1.6946 - acc: 0.4272 - val_loss: 1.6296 - val_acc: 0.4352
Epoch 4/30
150/150 [==============================] - 36s 243ms/step - loss: 1.6226 - acc: 0.4333 - val_loss: 1.5487 - val_acc: 0.4502
Epoch 5/30
150/150 [==============================] - 34s 230ms/step - loss: 1.5324 - acc: 0.4476 - val_loss: 1.4960 - val_acc: 0.4534
Epoch 6/30
150/150 [==============================] - 36s 242ms/step - loss: 1.4784 - acc: 0.4542 - val_loss: 1.4215 - val_acc: 0.4936
Epoch 7/30
150/150 [==============================] - 37s 246ms/step - loss: 1.4451 - acc: 0.4792 - val_loss: 1.3964 - val_acc: 0.4976
Epoch 8/30
150/150 [==============================] - 3

In [ ]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 0.9263944029808044
Test Accuracy: 0.6776831746101379


In [ ]:
pred = np.argmax(model1.predict(X_test), axis=-1)
print(classification_report(label_test, pred))

              precision    recall  f1-score   support

           0       0.56      0.61      0.58       314
           1       0.59      0.68      0.63      1184
           2       0.64      0.34      0.44       701
           3       0.75      0.87      0.81      2434
           4       0.39      0.24      0.30       470
           5       0.67      0.56      0.61       381
           6       0.64      0.73      0.68       231
           7       0.95      0.81      0.87       276

    accuracy                           0.68      5991
   macro avg       0.65      0.60      0.62      5991
weighted avg       0.67      0.68      0.66      5991



In [ ]:
#----------------------------------------------------------------------------------

In [ ]:
model2 = Sequential()
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model2.add(embedding_layer)

model2.add(LSTM(1024))

model2.add(Dense(8, activation='softmax'))
model2.compile(optimizer='Adamax', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
history = model2.fit(X_train, label_train, batch_size=128, epochs=100, verbose=1, validation_split=0.2)

score = model2.evaluate(X_test, label_test, verbose=1)

Epoch 1/100
150/150 [==============================] - 20s 114ms/step - loss: 1.8880 - acc: 0.4021 - val_loss: 1.4875 - val_acc: 0.4586
Epoch 2/100
150/150 [==============================] - 17s 112ms/step - loss: 1.4631 - acc: 0.4649 - val_loss: 1.3197 - val_acc: 0.5291
Epoch 3/100
150/150 [==============================] - 17s 114ms/step - loss: 1.3454 - acc: 0.5251 - val_loss: 1.3265 - val_acc: 0.5320
Epoch 4/100
150/150 [==============================] - 17s 116ms/step - loss: 1.3035 - acc: 0.5398 - val_loss: 1.2712 - val_acc: 0.5452
Epoch 5/100
150/150 [==============================] - 18s 118ms/step - loss: 1.2557 - acc: 0.5473 - val_loss: 1.2342 - val_acc: 0.5602
Epoch 6/100
150/150 [==============================] - 18s 117ms/step - loss: 1.2458 - acc: 0.5520 - val_loss: 1.2402 - val_acc: 0.5591
Epoch 7/100
150/150 [==============================] - 17s 115ms/step - loss: 1.2228 - acc: 0.5671 - val_loss: 1.1918 - val_acc: 0.5786
Epoch 8/100
150/150 [===========================

In [ ]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 1.276484727859497
Test Accuracy: 0.7599732875823975


In [ ]:
pred = np.argmax(model2.predict(X_test), axis=-1)
print(classification_report(label_test, pred))

              precision    recall  f1-score   support

           0       0.59      0.67      0.63       314
           1       0.77      0.72      0.74      1184
           2       0.68      0.61      0.64       701
           3       0.84      0.87      0.86      2434
           4       0.54      0.60      0.57       470
           5       0.68      0.65      0.66       381
           6       0.76      0.78      0.77       231
           7       0.91      0.84      0.88       276

    accuracy                           0.76      5991
   macro avg       0.72      0.72      0.72      5991
weighted avg       0.76      0.76      0.76      5991



In [ ]:
#------------------------------------------------------

In [ ]:
model3 = Sequential()

embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model3.add(embedding_layer)


model3.add(Conv1D(1024, 1, activation='softsign'))
model3.add(Conv1D(1024, 1, activation='softsign'))
model3.add(GlobalMaxPooling1D())
model3.add(Dense(8, activation='softmax'))
model3.compile(optimizer='Adamax', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
history = model3.fit(X_train, label_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

score = model3.evaluate(X_test, label_test, verbose=1)

Epoch 1/10
150/150 [==============================] - 8s 41ms/step - loss: 1.4140 - acc: 0.5028 - val_loss: 0.9065 - val_acc: 0.6839
Epoch 2/10
150/150 [==============================] - 6s 37ms/step - loss: 0.8311 - acc: 0.7166 - val_loss: 0.7654 - val_acc: 0.7252
Epoch 3/10
150/150 [==============================] - 6s 37ms/step - loss: 0.7007 - acc: 0.7543 - val_loss: 0.7218 - val_acc: 0.7438
Epoch 4/10
150/150 [==============================] - 6s 38ms/step - loss: 0.6340 - acc: 0.7788 - val_loss: 0.6960 - val_acc: 0.7557
Epoch 5/10
150/150 [==============================] - 6s 38ms/step - loss: 0.5985 - acc: 0.7906 - val_loss: 0.6676 - val_acc: 0.7657
Epoch 6/10
150/150 [==============================] - 6s 38ms/step - loss: 0.5527 - acc: 0.8113 - val_loss: 0.6619 - val_acc: 0.7718
Epoch 7/10
150/150 [==============================] - 6s 38ms/step - loss: 0.5231 - acc: 0.8224 - val_loss: 0.6457 - val_acc: 0.7776
Epoch 8/10
150/150 [==============================] - 6s 37ms/step - 

In [ ]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 0.6809646487236023
Test Accuracy: 0.7781672477722168


In [ ]:
pred = np.argmax(model3.predict(X_test), axis=-1)
print(classification_report(label_test, pred))

              precision    recall  f1-score   support

           0       0.62      0.84      0.71       314
           1       0.81      0.67      0.73      1184
           2       0.68      0.66      0.67       701
           3       0.84      0.90      0.87      2434
           4       0.63      0.55      0.59       470
           5       0.67      0.69      0.68       381
           6       0.79      0.80      0.80       231
           7       0.94      0.88      0.91       276

    accuracy                           0.78      5991
   macro avg       0.75      0.75      0.74      5991
weighted avg       0.78      0.78      0.78      5991



In [ ]:
#--------------------------------------------------------------------------------------------